In [1]:
import os, subprocess
import json
import uproot3
import awkward as ak
import numpy as np
from coffea import processor, util, hist

from plotter import *

In [2]:
lumis = {}
lumis['2016'] = 35.9
lumis['2017'] = 41.5
lumis['2018'] = 59.9

with open('xsec.json') as f:
  xs = json.load(f)

with open('pmap.json') as f:
  pmap = json.load(f)

systematics = ['nominal',
               'jet_triggerUp','jet_triggerDown',
               'btagWeightUp','btagWeightDown','btagEffStatUp','btagEffStatDown',
               'UESUp','UESDown','JESUp','JESDown','JERUp','JERDown',
              ]

In [3]:
year = '2016'
nfiles = len(subprocess.getoutput("ls infiles-split/"+year+"*.json").split())
outsum = processor.dict_accumulator()

In [4]:
# Load all files
for n in range(1,nfiles+1):

    with open('infiles-split/'+year+'_'+str(n)+'.json') as f:
      infiles = json.load(f)
    
    filename = '/myeosdir/vbf-category/outfiles/'+year+'_'+str(n)+'.coffea'
    if os.path.isfile(filename):
        out = util.load(filename)
        outsum.add(out)
    else:
        print(n,infiles.keys())
        #print("File " + filename + " is missing")
        
scale_lumi = {k: xs[k] * 1000 *lumis[year] / w for k, w in outsum['sumw'].items()}
outsum['templates'].scale(scale_lumi, 'dataset')

1 dict_keys(['DYJetsToLL_M-50_HT-1200to2500_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
3 dict_keys(['DYJetsToLL_M-50_HT-2500toInf_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
5 dict_keys(['DYJetsToLL_M-50_HT-400to600_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
6 dict_keys(['DYJetsToLL_M-50_HT-400to600_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
7 dict_keys(['DYJetsToLL_M-50_HT-400to600_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
8 dict_keys(['DYJetsToLL_M-50_HT-400to600_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
9 dict_keys(['DYJetsToLL_M-50_HT-400to600_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
10 dict_keys(['DYJetsToLL_M-50_HT-400to600_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
11 dict_keys(['DYJetsToLL_M-50_HT-400to600_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
13 dict_keys(['DYJetsToLL_M-50_HT-600to800_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
14 dict_keys(['DYJetsToLL_M-50_HT-600to800_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
15 dict_keys(['DYJetsToLL_M-50_HT-600to800_TuneCUETP8M1_13TeV-madgrap

580 dict_keys(['TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8'])
581 dict_keys(['TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8'])
582 dict_keys(['TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8'])
583 dict_keys(['TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8'])
584 dict_keys(['TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8'])
585 dict_keys(['TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8'])
586 dict_keys(['TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8'])
587 dict_keys(['TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8'])
588 dict_keys(['TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8'])
589 dict_keys(['TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8'])
590 dict_keys(['TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8'])
591 dict_keys(['TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8'])
592 dict_keys(['TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8'])
593 dict_keys(['TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8'])
594 dict_keys(['TTTo2L2Nu_TuneCP5_PSweights_13Te

719 dict_keys(['VBFHToBB_M-125_13TeV_powheg_pythia8_weightfix'])
721 dict_keys(['WJetsToLNu_HT-1200To2500_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
722 dict_keys(['WJetsToLNu_HT-1200To2500_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
723 dict_keys(['WJetsToLNu_HT-1200To2500_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
724 dict_keys(['WJetsToLNu_HT-1200To2500_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
725 dict_keys(['WJetsToLNu_HT-1200To2500_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
727 dict_keys(['WJetsToLNu_HT-200To400_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
728 dict_keys(['WJetsToLNu_HT-200To400_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
729 dict_keys(['WJetsToLNu_HT-200To400_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
730 dict_keys(['WJetsToLNu_HT-200To400_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
731 dict_keys(['WJetsToLNu_HT-200To400_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
732 dict_keys(['WJetsToLNu_HT-200To400_TuneCUETP8M1_13TeV-madgraphMLM-pythia8'])
733 dict_keys(['WJetsToLNu_HT-200T

In [5]:
templates = outsum['templates'].group('dataset', hist.Cat('process', 'Process'), pmap)
templates.sum('pt1','msd1','ddb1').integrate('region','signal').integrate('systematic', 'nominal').values()

/opt/conda/lib/python3.8/site-packages/coffea/hist/hist_tools.py:347: RuntimeWarning: Not all requested indices present in <Cat (name=dataset) instance at 0x7f672e9567f0>
  warnings.warn("Not all requested indices present in %r" % self, RuntimeWarning)


{('ZH',): array(0.32828541),
 ('WH',): array(0.47368026),
 ('QCD',): array(29490.20783253),
 ('Wjets',): array(124289.18508488),
 ('ttbar',): array(17731.79247547),
 ('data',): array(5253871.),
 ('muondata',): array(91381.)}

In [6]:
ptbins = [450, 500, 550, 600, 675, 800, 1200]
os.system('rm '+year+'/inclusive/signalregion.root')
fout = uproot3.create(year+'/inclusive/signalregion.root')
for i,b in enumerate(ptbins[:-1]):
    for p in pmap.keys():  
        print(p)
        for s in systematics:
            h = templates.integrate('region', 'signal').integrate('systematic',s).integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
            fout[p+"_pass_pt"+str(i+1)+"_"+s] = hist.export1d(h)
            h = templates.integrate('region', 'signal').integrate('systematic',s).integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
            fout[p+"_fail_pt"+str(i+1)+"_"+s] = hist.export1d(h)

fout.close()

ZH
WH
ttH


KeyError: ()

In [ ]:
os.system('rm '+year+'/inclusive/muonCR.root')
fout = uproot3.create(year+'/inclusive/muonCR.root')
for p in pmap.keys():  
    print(p)
    for s in systematics:
        # muonkin
        h = templates.integrate('region', 'muoncontrol').integrate('systematic',s).sum('pt1').integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
        fout[p+"_pass_"+s] = hist.export1d(h)
        h = templates.integrate('region', 'muoncontrol').integrate('systematic',s).sum('pt1').integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
        fout[p+"_fail_"+s] = hist.export1d(h)

In [ ]:
muonkin = outsum['muonkin'].group('dataset', hist.Cat('process', 'Process'), pmap).integrate('systematic','nominal')
mujetkin = outsum['mujetkin'].group('dataset', hist.Cat('process', 'Process'), pmap).integrate('systematic','nominal')

In [ ]:
if( 0 )
    os.system('rm '+year+'/inclusive/kin_muonCR.root')
    fout = uproot3.create(year+'/inclusive/kin_muonCR.root')
    for p in pmap.keys():  
        print(p)
    
        # muonkin
        h = muonkin.integrate('region', 'muoncontrol').sum('ptmu','etamu').integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
        fout[p+"_pass_msd1"] = hist.export1d(h)
        h = muonkin.integrate('region', 'muoncontrol').sum('ptmu','etamu').integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
        fout[p+"_fail_msd1"] = hist.export1d(h)
        h = muonkin.integrate('region', 'muoncontrol').sum('ptmu','etamu','ddb1').integrate('process',p)
        fout[p+"_msd1"] = hist.export1d(h)
        
        h = muonkin.integrate('region', 'muoncontrol').sum('msd1','etamu').integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
        fout[p+"_pass_ptmu"] = hist.export1d(h)
        h = muonkin.integrate('region', 'muoncontrol').sum('msd1','etamu').integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
        fout[p+"_fail_ptmu"] = hist.export1d(h)
        h = muonkin.integrate('region', 'muoncontrol').sum('msd1','etamu','ddb1').integrate('process',p)
        fout[p+"_ptmu"] = hist.export1d(h)
    
        h = muonkin.integrate('region', 'muoncontrol').sum('msd1','ptmu').integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
        fout[p+"_pass_etamu"] = hist.export1d(h)
        h = muonkin.integrate('region', 'muoncontrol').sum('msd1','ptmu').integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
        fout[p+"_fail_etamu"] = hist.export1d(h)
        h = muonkin.integrate('region', 'muoncontrol').sum('msd1','ptmu','ddb1').integrate('process',p)
        fout[p+"_etamu"] = hist.export1d(h)
    
        # mujetkin
        h = mujetkin.integrate('region', 'muoncontrol').sum('msd1','eta1','ddb1').integrate('process',p)
        fout[p+"_pt1"] = hist.export1d(h)
        h = mujetkin.integrate('region', 'muoncontrol').sum('msd1','pt1','ddb1').integrate('process',p)
        fout[p+"_eta1"] = hist.export1d(h)
        h = mujetkin.integrate('region', 'muoncontrol').sum('msd1','pt1','eta1').integrate('process',p)
        fout[p+"_ddb1"] = hist.export1d(h)

fout.close()